<img src="raw\profile_manoelgadi.png" width=100 height=100 align="right">

Author: Manoel Gadi

Last revision: 19/July/2020

---

Welcome to the GROUP VIDEO starter code.

Your group task is to record a video of 10 - 15 minutes CLEANING DATA and DEVELOPING 2 MODELS using the RBA_KYC_Accounts_ALL_Ids.xlsx dataset. You should use Pandas, Matplotlib and Scikit Learn (Sklearn) libraries only if possible. Your video should include:

* A quick explanation of Pandas, Matplotlib and Sklearn, specially including their histories. (less than a minute)
* A quick Exploratory Data Analysis of the data set used.
* Using the EDA, explain the problems of data quality in this data set.
* Research about data preparation with Python. Propose and implement ways to clean, input missing and prepare the data for modelling. (YOUR VIDEO SHOULD FOCUS IN THIS PART - 70-80% OF THE VIDEO SHOULD BE ABOUT DATA CLEANING AND DATA PREPARATION)
* Then, develop one Regression model using rbaValue as target variable.  Remember to assess the model performance using an appropriate measure for regression. 
* Then, develop one binary Classification model using Risk_Description setting Risk_Description = Low as 0 and else 1 (Medium or High). Remember to assess your model performance using a appropriate measure for classification.

ps. please, don't improve the model, you don't need to make good one, simply do a quick one! This exercise is about explaining, in a video lecture, how to prepare the data for modelling , not about making a good model .

ps. You can apply any of the steps in the model development seing in the last class, but be coincious of time it may take to do it all, so avoid overcomplicating things.

Please find below couple videos from previous years (different data set and different objective) and from which you'll also learn some Python ideas:

* 2018 MBD01 Group E Variable Transformation - https://www.youtube.com/watch?v=j_0HQ2UF_L0
* 2019 - Risk & Fraud Analytics - Scikit Learn & Lending Club - Group C - https://www.youtube.com/watch?v=oTmHMXsksM8

---

### Importing the database: RBA_KYC_Accounts_ALL_Ids.xlsx

In [27]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn_pandas import CategoricalImputer
from sklearn.preprocessing import StandardScaler

In [28]:
# df = pd.read_excel('raw\RBA_KYC_Accounts_ALL_Ids.xlsx')

# For speed, we store the Excel file in a Pickle
# https://stackoverflow.com/a/17098736/3780957
# df.to_pickle('storage\RBA_KYC_Accounts_ALL_Ids.pkl')

In [29]:
df = pd.read_pickle('storage\RBA_KYC_Accounts_ALL_Ids.pkl')

In [30]:
df['Risk_Description'] = df['rba_grade_desc']
df.drop(['rba_grade_desc'], axis = 1, inplace = True) 

## Checking some quick statistics

In [31]:
df.describe()

,org_code,branchCode,customerId,customerType,residentStatus,age_in_year,birthCountry,profession,companyType,lastUpdate,...,number_of_cash_dep_90_days,avg_cash_deposit_90_days,number_of_deposit_90_days,avg_of_dep_90_days,number_of_cash_wd_90_days,avg_of_cash_wd_90_days,number_of_wd_90_days,avg_of_wd_90_days,score_card_Desc,rbaValue
count,224868.0,224868.000000,2.248680e+05,224868.000000,224868.000000,220153.000000,0.0,79748.000000,114574.000000,0.0,...,224868.000000,2.248680e+05,224868.000000,2.248680e+05,224868.000000,2.248680e+05,224868.000000,2.248680e+05,0.0,224868.000000
mean,1.0,448.451323,4.394503e+16,1.114418,1.021221,41.310575,NaN,18.309487,49.857577,NaN,...,0.491422,-1.113435e+03,1.131437,-3.540383e+03,0.254305,4.763851e+02,0.868518,2.450953e+03,NaN,17.624194
std,0.0,228.193308,2.349321e+16,0.318319,0.144122,16.000536,NaN,6.586481,6.736115,NaN,...,5.335240,3.028084e+04,19.713739,9.151602e+04,1.175586,2.028110e+04,12.373992,5.819818e+04,NaN,8.368345
min,1.0,201.000000,2.020028e+09,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,NaN,...,0.000000,-5.482044e+06,0.000000,-1.654371e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,NaN,4.080000
25%,1.0,208.000000,2.070207e+16,1.000000,1.000000,31.000000,NaN,20.000000,51.000000,NaN,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,NaN,13.330000
50%,1.0,401.000000,4.011205e+16,1.000000,1.000000,41.000000,NaN,20.000000,51.000000,NaN,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,NaN,16.530000
75%,1.0,701.000000,7.012410e+16,1.000000,1.000000,52.000000,NaN,21.000000,51.000000,NaN,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,NaN,21.330000
max,1.0,805.000000,8.052929e+16,2.000000,2.000000,119.000000,NaN,39.000000,100.000000,NaN,...,1267.000000,0.000000e+00,3292.000000,5.867291e+05,78.000000,8.394000e+06,2858.000000,9.942530e+06,NaN,100.000000


## Creating the variable creationDate_indays

In [32]:
df['creationDate_indays'] = df['creationDate'].astype('datetime64[ns]').max()-df['creationDate'].astype('datetime64[ns]') 
df['creationDate_indays'].head()

0   1295 days
1   1860 days
2   1860 days
3   1860 days
4   1860 days
Name: creationDate_indays, dtype: timedelta64[ns]

## Creating the variable date of birth

In [33]:
def f(end):
    global max_date
    try:
        r = relativedelta(max_date, end) 
        return(r.years)
    except:
        return(0)
#    r = relativedelta(pd.to_datetime('now'), end) 
#    return '{} years {} days'.format(r.years, r.days)

In [34]:
max_date = df['dateOfBirth'].astype('datetime64[ns]').max()
df['age'] = df['dateOfBirth'].astype('datetime64[ns]').apply(f)
df['age'].head()

0    0
1    0
2    0
3    0
4    0
Name: age, dtype: int64

## Checking for percentage (%) of missing values

In [35]:
100*df.isnull().sum()/df.isnull().count()

org_code                        0.000000
branchCode                      0.000000
customerId                      0.000000
fullaccount                     0.000000
customerType                    0.000000
creationDate                    0.000000
onboarding                      0.000000
residentStatus                  0.000000
residentCountry                 0.265044
nationalityOriginal             1.529342
extraNationality               99.202643
dateOfBirth                     2.096786
age_in_year                     2.096786
birthCountry                  100.000000
birthPlace                      9.713254
profession                     64.535639
companyType                    49.048331
giinCode                       99.994664
lastUpdate                    100.000000
status                          0.000000
ledgerCode                      0.000000
accountCurrency                 0.000000
jointAccount                    0.000000
IsBlackListed                   0.000000
CUS_CLASS       

## Making some histograms

In [36]:
# %matplotlib inline
# import matplotlib.pylab as plt
# attribute_columns = list(df.columns)
# if True:
#     fontsize = 10
#     fig = df[attribute_columns].hist(bins = 20, figsize = (12, 12),
#                                        xlabelsize=fontsize,
#                                        ylabelsize=fontsize)
#     for x in fig.ravel():
#         x.title.set_size(fontsize)
# else:
#     nrows = (len(attribute_columns)+1) // 2
#     plt.figure(figsize=(12,25))
#     for i in range(len(attribute_columns)):
#         name = attribute_columns[i]
#         plt.subplot(nrows,2,i+1)
#         data[name].hist()
#         plt.title(name, fontsize=14)
    
# plt.show()

### Now, is your group turn: 
...

---

# Introduction

A quick explanation of Pandas, Matplotlib and Sklearn, specially including their histories. (less than a minute)
https://medium.com/personal-project/numpy-pandas-and-scikit-learn-explained-e7336baecedc

## Pandas
https://en.wikipedia.org/wiki/Pandas_(software)#History
Developer Wes McKinney started working on pandas in 2008 while at AQR Capital Management out of the need for a high performance, flexible tool to perform quantitative analysis on financial data. Before leaving AQR he was able to convince management to allow him to open source the library.
Another AQR employee, Chang She, joined the effort in 2012 as the second major contributor to the library.
In 2015, pandas signed on as a fiscally sponsored project of NumFOCUS, a 501(c)(3) nonprofit charity in the United States.[10]

## Matplotlib
https://en.wikipedia.org/wiki/Matplotlib
Matplotlib was originally written by John D. Hunter, since then it has an active development community,[4] and is distributed under a BSD-style license. Michael Droettboom was nominated as matplotlib's lead developer shortly before John Hunter's death in August 2012,[5] and further joined by Thomas Caswell.[6][7]


## Sklearn
https://en.wikipedia.org/wiki/Scikit-learn
The scikit-learn project started as scikits.learn, a Google Summer of Code project by David Cournapeau. Its name stems from the notion that it is a "SciKit" (SciPy Toolkit), a separately-developed and distributed third-party extension to SciPy.[4] The original codebase was later rewritten by other developers. In 2010 Fabian Pedregosa, Gael Varoquaux, Alexandre Gramfort and Vincent Michel, all from the French Institute for Research in Computer Science and Automation in Rocquencourt, France, took leadership of the project and made the first public release on February the 1st 2010.[5] Of the various scikits, scikit-learn as well as scikit-image were described as "well-maintained and popular" in November 2012.[6] Scikit-learn is one of the most popular machine learning libraries on GitHub.[7]

# EDA

In [37]:
# Datatime conversion

df['creationDate'] = df['creationDate'].astype('datetime64[ns]')
df['dateOfBirth'] = df['dateOfBirth'].astype('datetime64[ns]')
df['date_of_assessment'] = df['date_of_assessment'].astype('datetime64[ns]')

In [38]:
df.head()

,org_code,branchCode,customerId,fullaccount,customerType,creationDate,onboarding,residentStatus,residentCountry,nationalityOriginal,...,number_of_wd_90_days,avg_of_wd_90_days,date_of_assessment,rbaGradeAbrv,score_card,score_card_Desc,rbaValue,Risk_Description,creationDate_indays,age
0,1,201,20100400550000072,020100400550015817000,2,2015-04-05,N,1,JO,JO,...,0,0.0,2019-01-24,L,E-Existing,NaN,13.20,Low,1295 days,0
1,1,202,20200207860000638,020200207860011330000,1,2013-09-17,N,1,JO,JO,...,0,0.0,2019-01-24,L,I-Existing,NaN,16.93,Low,1860 days,0
2,1,202,20200207860000638,020200207860011389002,1,2013-09-17,N,1,JO,JO,...,0,0.0,2019-01-24,L,I-Existing,NaN,16.93,Low,1860 days,0
3,1,202,20200207860000638,020200207860010060000,1,2013-09-17,N,1,JO,JO,...,1,50.0,2019-01-24,L,I-Existing,NaN,16.93,Low,1860 days,0
4,1,202,20200207860000638,020200207860010010000,1,2013-09-17,N,1,JO,JO,...,1,56.0,2019-01-24,L,I-Existing,NaN,16.93,Low,1860 days,0


## Description per type

In [39]:
df.describe(include='object')

,fullaccount,onboarding,residentCountry,nationalityOriginal,extraNationality,birthPlace,giinCode,status,accountCurrency,rbaGradeAbrv,score_card,Risk_Description
count,224868,224868,224272,221429,1793,203026,12,224868,224868,224868,224868,224868
unique,220434,2,62,67,35,1000,1,1,20,3,4,3
top,020902507770010010000,N,JO,JO,US,????,ZLVK2E.99999.SL.400,Active,JOD,L,I-Existing,Low
freq,26,219791,219253,213141,1028,76312,12,224868,214657,159680,178495,159680


In [40]:
df.describe(include=np.number)

,org_code,branchCode,customerId,customerType,residentStatus,age_in_year,birthCountry,profession,companyType,lastUpdate,...,number_of_deposit_90_days,avg_of_dep_90_days,number_of_cash_wd_90_days,avg_of_cash_wd_90_days,number_of_wd_90_days,avg_of_wd_90_days,score_card_Desc,rbaValue,creationDate_indays,age
count,224868.0,224868.000000,2.248680e+05,224868.000000,224868.000000,220153.000000,0.0,79748.000000,114574.000000,0.0,...,224868.000000,2.248680e+05,224868.000000,2.248680e+05,224868.000000,2.248680e+05,0.0,224868.000000,224868,224868.0
mean,1.0,448.451323,4.394503e+16,1.114418,1.021221,41.310575,NaN,18.309487,49.857577,NaN,...,1.131437,-3.540383e+03,0.254305,4.763851e+02,0.868518,2.450953e+03,NaN,17.624194,2419 days 01:19:10.178785,0.0
std,0.0,228.193308,2.349321e+16,0.318319,0.144122,16.000536,NaN,6.586481,6.736115,NaN,...,19.713739,9.151602e+04,1.175586,2.028110e+04,12.373992,5.819818e+04,NaN,8.368345,2410 days 07:14:32.396639,0.0
min,1.0,201.000000,2.020028e+09,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,NaN,...,0.000000,-1.654371e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,NaN,4.080000,0 days 00:00:00,0.0
25%,1.0,208.000000,2.070207e+16,1.000000,1.000000,31.000000,NaN,20.000000,51.000000,NaN,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,NaN,13.330000,731 days 00:00:00,0.0
50%,1.0,401.000000,4.011205e+16,1.000000,1.000000,41.000000,NaN,20.000000,51.000000,NaN,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,NaN,16.530000,1739 days 00:00:00,0.0
75%,1.0,701.000000,7.012410e+16,1.000000,1.000000,52.000000,NaN,21.000000,51.000000,NaN,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,NaN,21.330000,3065 days 00:00:00,0.0
max,1.0,805.000000,8.052929e+16,2.000000,2.000000,119.000000,NaN,39.000000,100.000000,NaN,...,3292.000000,5.867291e+05,78.000000,8.394000e+06,2858.000000,9.942530e+06,NaN,100.000000,14644 days 00:00:00,0.0


In [41]:
df.describe(include=np.datetime64)

,creationDate,dateOfBirth,date_of_assessment
count,224868,220153,224868
unique,7041,21457,1
top,2013-11-20 00:00:00,1992-07-22 00:00:00,2019-01-24 00:00:00
freq,1079,1035,224868
first,1978-09-17 00:00:00,1900-06-01 00:00:00,2019-01-24 00:00:00
last,2018-10-21 00:00:00,2018-10-15 00:00:00,2019-01-24 00:00:00


## Remove fully NA columns

In [42]:
# df.info()

In [43]:
# Drop columns where all the values are missing or null
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html
df.dropna(axis='columns', how='all', inplace=True)

In [44]:
# df.info()
# 100*df.isnull().sum()/df.isnull().count()

## Imputation and Scaling

There are [several ways of imputing missing values](https://scikit-learn.org/stable/modules/impute.html#multivariate-feature-imputation). I'd like to use a multivariate approach, which is much better than univariate methods (imputing by the mean, etc.). To do so, I will use the methods available in `scikit-learn` library ([here](https://scikit-learn.org/stable/modules/impute.html#multivariate-feature-imputation)).

In [45]:
df.select_dtypes(include=np.number)

,org_code,branchCode,customerId,customerType,residentStatus,age_in_year,profession,companyType,ledgerCode,jointAccount,...,avg_cash_deposit_90_days,number_of_deposit_90_days,avg_of_dep_90_days,number_of_cash_wd_90_days,avg_of_cash_wd_90_days,number_of_wd_90_days,avg_of_wd_90_days,rbaValue,creationDate_indays,age
0,1,201,20100400550000072,2,1,NaN,NaN,35.0,5817,0,...,0.0,0,0.0,0,0.0,0,0.0,13.20,1295 days,0
1,1,202,20200207860000638,1,1,30.0,27.0,NaN,1330,0,...,0.0,0,0.0,0,0.0,0,0.0,16.93,1860 days,0
2,1,202,20200207860000638,1,1,30.0,27.0,NaN,1389,0,...,0.0,0,0.0,0,0.0,0,0.0,16.93,1860 days,0
3,1,202,20200207860000638,1,1,30.0,27.0,NaN,60,0,...,-550.0,4,-550.0,1,50.0,1,50.0,16.93,1860 days,0
4,1,202,20200207860000638,1,1,30.0,27.0,NaN,10,0,...,-540.0,4,-540.0,1,56.0,1,56.0,16.93,1860 days,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224863,1,404,40400811920003507,1,1,66.0,NaN,NaN,20,0,...,0.0,0,0.0,0,0.0,0,0.0,16.53,364 days,0
224864,1,202,20200577640010547,2,1,13.0,NaN,35.0,5300,0,...,0.0,0,0.0,0,0.0,0,0.0,57.60,5199 days,0
224865,1,502,50220271300005255,1,1,41.0,NaN,51.0,2980,0,...,0.0,0,0.0,0,0.0,0,0.0,14.53,6633 days,0
224866,1,704,70424500690004664,1,1,70.0,NaN,NaN,1396,0,...,0.0,0,0.0,0,0.0,0,0.0,24.53,3931 days,0


https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html

A more sophisticated approach is to use the IterativeImputer class, which models each feature with missing values as a function of other features, and uses that estimate for imputation. It does so in an iterated round-robin fashion: at each step, a feature column is designated as output y and the other feature columns are treated as inputs X. A regressor is fit on (X, y) for known y. Then, the regressor is used to predict the missing values of y. This is done for each feature in an iterative fashion, and then is repeated for max_iter imputation rounds. The results of the final imputation round are returned.

In [46]:
num_imputer = IterativeImputer(max_iter=10, random_state=0)
num_columns = df.select_dtypes(include=['int64', 'float64']).columns
num_feats = num_imputer.fit_transform(df[num_columns])
num_feats = pd.DataFrame(data=num_feats, columns=num_columns)

To perform imputaton on categorical variables, I decided to use a method provided by the library `sklearn_pandas` that implements a `CategoricalImputer`.

https://github.com/scikit-learn-contrib/sklearn-pandas

Since the scikit-learn Imputer transformer currently only works with numbers, sklearn-pandas provides an equivalent helper transformer that works with strings, substituting null values with the most frequent value in that column. Alternatively, you can specify a fixed value to use.

Alternative
https://stackoverflow.com/questions/42846345/sklearn-categorical-imputer

In [47]:


cat_imputer = CategoricalImputer()
cat_feats = pd.DataFrame()
for col in df.select_dtypes(include=['object', 'datetime64']):
    cat_feats[col] = cat_imputer.fit_transform(df[col])

Feature engineering for the dates

In [48]:
df_time = pd.DataFrame()
for col in df2.select_dtypes(include=['datetime64']):
    df_time['{}_year'.format(col)] = df2[col].dt.year
    df_time['{}_month'.format(col)] = df2[col].dt.month
    df_time['{}_day'.format(col)] = df2[col].dt.day
    df_time['{}_dayofyear'.format(col)] = df2[col].dt.dayofyear

Finally, the result of imputing both: numerical and categorical, is now merged together in a single Dataset (or you can also use a dataframe, if you prefer).

In [49]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
# date_columns = df.select_dtypes(include=['datetime64']).columns
df2 = pd.concat([cat_feats, num_feats, df_time], axis=1, sort=False)

Scale all the numeric values, at this point.

In [50]:
# https://stackoverflow.com/a/36475297/3780957

num_columns = df2.select_dtypes(include=['int64', 'float64']).columns

scaler = StandardScaler()
df2[num_columns] = scaler.fit_transform(df2[num_columns])

Now, my dataset is ready to be analyzed. Lots of categorical and numerical variables.

In [51]:
df2.describe()

,org_code,branchCode,customerId,customerType,residentStatus,age_in_year,profession,companyType,ledgerCode,jointAccount,...,creationDate_day,creationDate_dayofyear,dateOfBirth_year,dateOfBirth_month,dateOfBirth_day,dateOfBirth_dayofyear,date_of_assessment_year,date_of_assessment_month,date_of_assessment_day,date_of_assessment_dayofyear
count,224868.0,2.248680e+05,2.248680e+05,2.248680e+05,2.248680e+05,2.248680e+05,2.248680e+05,2.248680e+05,2.248680e+05,2.248680e+05,...,2.248680e+05,2.248680e+05,2.248680e+05,2.248680e+05,2.248680e+05,2.248680e+05,224868.0,224868.0,224868.0,224868.0
mean,0.0,-3.367415e-13,4.082475e-14,-1.266870e-13,1.632239e-13,-5.844394e-15,-4.442901e-15,-1.489806e-14,5.498509e-15,2.906594e-15,...,1.906912e-16,-4.154938e-16,3.810405e-15,4.755211e-15,-1.591429e-15,8.384576e-16,0.0,0.0,0.0,0.0
std,0.0,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,...,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,0.0,0.0,0.0,0.0
min,0.0,-1.084396e+00,-1.870545e+00,-3.594457e-01,-1.472462e-01,-9.010643e+00,-1.581392e+01,-6.525250e+00,-8.192297e-01,-1.487680e-01,...,-1.681182e+00,-1.735944e+00,-4.885310e+00,-1.458979e+00,-1.385167e+00,-1.545371e+00,0.0,0.0,0.0,0.0
25%,0.0,-1.053720e+00,-9.893499e-01,-3.594457e-01,-1.472462e-01,-6.485461e-01,-8.638835e-02,-9.899618e-01,-8.114225e-01,-1.487680e-01,...,-8.918733e-01,-8.764719e-01,-6.883883e-01,-8.934170e-01,-9.533272e-01,-8.786293e-01,0.0,0.0,0.0,0.0
50%,0.0,-2.079440e-01,-1.631530e-01,-3.594457e-01,-1.472462e-01,-1.794576e-02,1.144389e-01,1.184785e-01,1.894578e-01,-1.487680e-01,...,1.019383e-02,-7.342251e-03,6.329915e-02,-4.507408e-02,1.831309e-02,7.315436e-03,0.0,0.0,0.0,0.0
75%,0.0,1.106734e+00,1.114327e+00,-3.594457e-01,-1.472462e-01,6.757146e-01,2.639956e-01,9.910813e-01,2.628453e-01,-1.487680e-01,...,9.122610e-01,8.521304e-01,6.897053e-01,8.032688e-01,8.819933e-01,8.293260e-01,0.0,0.0,0.0,0.0
max,0.0,1.562489e+00,1.557230e+00,2.782061e+00,6.791346e+00,4.900737e+00,6.521388e+01,7.995029e+00,5.965209e+00,6.721876e+00,...,1.701570e+00,1.788859e+00,2.506283e+00,1.651612e+00,1.853634e+00,1.788338e+00,0.0,0.0,0.0,0.0


In [52]:
100*df2.isnull().sum()/df2.isnull().count()


fullaccount                     0.0
creationDate                    0.0
onboarding                      0.0
residentCountry                 0.0
nationalityOriginal             0.0
extraNationality                0.0
dateOfBirth                     0.0
birthPlace                      0.0
giinCode                        0.0
status                          0.0
accountCurrency                 0.0
date_of_assessment              0.0
rbaGradeAbrv                    0.0
score_card                      0.0
Risk_Description                0.0
org_code                        0.0
branchCode                      0.0
customerId                      0.0
customerType                    0.0
residentStatus                  0.0
age_in_year                     0.0
profession                      0.0
companyType                     0.0
ledgerCode                      0.0
jointAccount                    0.0
IsBlackListed                   0.0
CUS_CLASS                       0.0
ECO_SEC                     